# 🔥 BLIP(ViT-L) 하이퍼파라미터 최적화 with Optuna

이 노트북에서 **Optuna TPE 알고리즘**을 사용하여 BLIP 모델의 최적 하이퍼파라미터를 찾습니다.

## 🎯 **Optuna 장점:**
- ✅ **로컬 실행**: 네트워크 타임아웃 없음
- ✅ **TPE 알고리즘**: Bayesian Optimization보다 효율적
- ✅ **빠르고 안정적**: 설정 간단, 오류 적음
- ✅ **스마트한 최적화**: 적은 실험으로 더 좋은 결과

## 📋 튜닝할 하이퍼파라미터
- **학습률 (Learning Rate)**: 1e-6 ~ 5e-5 (log scale)
- **배치 크기 (Batch Size)**: 4, 8, 16
- **에포크 수 (Epochs)**: 2, 3, 4
- **가중치 감쇠 (Weight Decay)**: 0.01 ~ 0.1 (log scale)
- **Warmup Steps**: 100, 300, 500, 800, 1000

## 🚀 실행 방법
1. **환경 확인**: GPU, 라이브러리 버전 체크
2. **Optuna 최적화 실행**: 10개 Trial 자동 실험 (60-90분)
3. **결과 자동 저장**: 최적 파라미터 JSON 파일 저장
4. **최종 학습**: 최적 파라미터로 전체 데이터셋 학습


## 🔧 1. 환경 설정 및 확인


In [ ]:
# 필수 라이브러리 import
import os
import sys
import subprocess
import torch
import transformers
import optuna
import json
import re
from datetime import datetime

print("🔥 BLIP Optuna 하이퍼파라미터 최적화!")
print("=" * 50)
print(f"✅ PyTorch 버전: {torch.__version__}")
print(f"✅ CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU 메모리: {torch.cuda.get_device_properties(0).total_memory // 1024**3}GB")
print(f"✅ Transformers 버전: {transformers.__version__}")
print(f"✅ Optuna 버전: {optuna.__version__}")
print(f"✅ 현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

# 📁 경로 확인
blip_code_path = "./BLIP_ViT_L_Finetuning"
train_script = os.path.join(blip_code_path, "blip_finetune.py")
dataset_path = "./dataset/VQAv2"

print("📁 경로 확인:")
print(f"  - BLIP 코드 폴더: {blip_code_path} {'✅' if os.path.exists(blip_code_path) else '❌'}")
print(f"  - 학습 스크립트: {train_script} {'✅' if os.path.exists(train_script) else '❌'}")
print(f"  - 데이터셋: {dataset_path} {'✅' if os.path.exists(dataset_path) else '❌'}")

# 데이터 파일 상세 확인
train_file = os.path.join(dataset_path, "train.json")
val_file = os.path.join(dataset_path, "val.json")
print(f"  - 학습 데이터: {train_file} {'✅' if os.path.exists(train_file) else '❌'}")
print(f"  - 검증 데이터: {val_file} {'✅' if os.path.exists(val_file) else '❌'}")
print("=" * 50)


🔥 BLIP Optuna 하이퍼파라미터 최적화!
✅ PyTorch 버전: 2.7.1+cu126
✅ CUDA 사용 가능: True
✅ GPU 이름: NVIDIA GeForce RTX 4080 Laptop GPU
✅ GPU 메모리: 11GB
✅ Transformers 버전: 4.53.1
✅ Optuna 버전: 4.4.0
✅ 현재 시간: 2025-07-21 17:01:05
📁 경로 확인:
  - BLIP 코드 폴더: ./BLIP_ViT_L_Finetuning ✅
  - 학습 스크립트: ./BLIP_ViT_L_Finetuning\blip_finetune.py ✅
  - 데이터셋: ./dataset/VQAv2 ✅
  - 학습 데이터: ./dataset/VQAv2\train.json ✅
  - 검증 데이터: ./dataset/VQAv2\val.json ✅


## 🔥 2. Optuna 하이퍼파라미터 최적화 함수


In [ ]:
# Optuna 최적화 함수

def extract_eval_loss_from_output(output_text):
    """수정된 eval_loss 추출 함수 - 올바른 파싱"""
    lines = output_text.split('\n')  # 실제 개행문자로 분할
    eval_loss = None
    
    # 다양한 패턴으로 eval_loss 찾기
    for line in lines:
        if 'eval_loss' in line:
            try:
                # 패턴 1: {'eval_loss': 2.3456} 또는 "eval_loss": 2.3456
                match1 = re.search(r"['\"]?eval_loss['\"]?\s*:\s*([0-9]+\.?[0-9]*)", line)
                if match1:
                    eval_loss = float(match1.group(1))
                    print(f"✅ eval_loss 찾음: {eval_loss}")
                    break
                    
                # 패턴 2: eval_loss=2.3456
                match2 = re.search(r"eval_loss\s*=\s*([0-9]+\.?[0-9]*)", line)
                if match2:
                    eval_loss = float(match2.group(1))
                    print(f"✅ eval_loss 찾음: {eval_loss}")
                    break
                    
            except (ValueError, AttributeError):
                continue
    
    if eval_loss is None:
        print("❌ eval_loss 파싱 실패 - 기본값 999.0 반환")
        return 999.0
    
    return eval_loss

def optuna_objective(trial):
    """Optuna 최적화 목적 함수 (WandB 완전 제거)"""
    
    # 🎯 하이퍼파라미터 제안 
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 5e-5, log=True)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    num_epochs = trial.suggest_int('num_epochs', 2, 4)
    weight_decay = trial.suggest_float('weight_decay', 0.01, 0.1, log=True)
    warmup_steps = trial.suggest_categorical('warmup_steps', [100, 300, 500, 800, 1000])
    warmup_ratio = warmup_steps / 2000  # warmup_steps를 ratio로 변환
    
    print(f"\\n🎯 Optuna Trial {trial.number + 1}:")
    print(f"  - learning_rate: {learning_rate:.2e}")
    print(f"  - batch_size: {batch_size}")
    print(f"  - num_epochs: {num_epochs}")
    print(f"  - weight_decay: {weight_decay:.4f}")
    print(f"  - warmup_steps: {warmup_steps} (ratio: {warmup_ratio:.3f})")
    print(f"  - 예상 시간: {num_epochs * 3}분")
    print("=" * 50)
    
    # 🚀 완전히 수정된 학습 실행 명령어
    cmd = [
        sys.executable, 
        "blip_finetune.py",
        "--train_file", "../dataset/VQAv2/train.json",
        "--val_file", "../dataset/VQAv2/val.json",
        "--max_train_samples", str(800),
        "--max_val_samples", str(400),
        "--per_device_train_batch_size", str(batch_size),
        "--per_device_eval_batch_size", str(batch_size),
        "--logging_steps", str(20),
        "--eval_strategy", "epoch",
        "--save_strategy", "no",
        "--load_best_model_at_end", "false",  # ✅ 수정된 blip_finetune.py와 호환
        "--num_train_epochs", str(num_epochs),
        "--output_dir", f"./optuna-trial-{trial.number}",
        "--learning_rate", str(learning_rate),
        "--weight_decay", str(weight_decay),
        "--warmup_ratio", str(warmup_ratio),
        # WandB 관련 인수 완전 제거
    ]
    
    print(f"🚀 실행 명령어:")
    print(f"{' '.join(cmd)}")
    print("⏰ GPU 학습 시작...")
    
    try:
        start_time = datetime.now()
        
        # 🚫 WandB 완전 비활성화 (환경변수)
        env = os.environ.copy()
        env["WANDB_MODE"] = "disabled"
        env["WANDB_DISABLED"] = "true"
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            cwd="./BLIP_ViT_L_Finetuning",
            env=env,
            timeout=36000
        )
        end_time = datetime.now()
        runtime = (end_time - start_time).total_seconds()
        
        if result.returncode == 0:
            # ✅ 성공 시 eval_loss 추출
            eval_loss = extract_eval_loss_from_output(result.stdout)
            
            print(f"✅ Trial {trial.number + 1} 성공!")
            print(f"📊 eval_loss: {eval_loss:.4f}")
            print(f"⏰ 실행 시간: {runtime:.1f}초")
            print("=" * 50)
            
            return eval_loss
            
        else:
            print(f"❌ Trial {trial.number + 1} 실패")
            print(f"Return code: {result.returncode}")
            print(f"STDERR: {result.stderr[:500]}...")
            return 999.0
            
    except subprocess.TimeoutExpired:
        print(f"⏰ Trial {trial.number + 1} 시간 초과 (1시간)")
        return 999.0
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} 오류: {e}")
        return 999.0

print("✅ Optuna 함수 정의 완료!")


✅ Optuna 함수 정의 완료!


## 🚀 3. Optuna 하이퍼파라미터 최적화 실행

**로컬 기반 안정적 최적화 (WandB 완전 제거)**

### 🎯 **설정:**
- **최적화 알고리즘**: TPE (Tree-structured Parzen Estimator)
- **실험 수**: 10개
- **샘플 수**: 800 (train) + 400 (val)
- **예상 시간**: 60-90분


In [ ]:
# 🔥 Optuna 하이퍼파라미터 최적화 실행 (완전히 정리된 버전)

try:
    # Optuna 스터디 생성
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),  # 재현 가능한 결과
        study_name=f"blip-vqa-final-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    
    print("🎯 Optuna 하이퍼파라미터 최적화 시작!")
    print(f"📊 스터디 이름: {study.study_name}")
    print("⏰ 10개 실험 순차 실행...")
    print("🔧 특징:")
    print("  ✅ WandB 완전 제거")
    print("  ✅ 모든 인수 오류 해결") 
    print("  ✅ 환경변수로 WandB 비활성화")
    print("  ✅ 빠른 로컬 실행")
    print("\\n" + "=" * 60)
    
    # 10개 Trial 실행
    study.optimize(optuna_objective, n_trials=10)
    
    print("\\n" + "=" * 60)
    print("🎉 Optuna 최적화 완료!")
    print("=" * 60)
    
    # 🏆 최적 결과 출력
    print(f"✅ 최적 eval_loss: {study.best_value:.4f}")
    print("\\n🎯 최적 하이퍼파라미터:")
    for key, value in study.best_params.items():
        if key == 'learning_rate':
            print(f"  - {key}: {value:.2e}")
        elif key in ['weight_decay']:
            print(f"  - {key}: {value:.4f}")
        else:
            print(f"  - {key}: {value}")
    
    print(f"\\n🏆 최적 Trial 번호: {study.best_trial.number + 1}")
    
    # 📊 전체 결과 요약
    print("\\n📊 전체 실험 결과 요약:")
    print("-" * 60)
    print(f"{'Trial':<8} {'eval_loss':<12} {'lr':<10} {'batch':<8} {'epochs':<8}")
    print("-" * 60)
    
    successful_trials = 0
    for trial in study.trials:
        if trial.value is not None and trial.value < 999.0:
            lr = trial.params.get('learning_rate', 0)
            batch = trial.params.get('batch_size', 0)
            epochs = trial.params.get('num_epochs', 0)
            print(f"{trial.number+1:<8} {trial.value:<12.4f} {lr:<10.2e} {batch:<8} {epochs:<8}")
            successful_trials += 1
    
    if successful_trials == 0:
        print("❌ 성공한 Trial이 없습니다. 아래 디버깅 정보를 확인하세요.")
        for trial in study.trials:
            if trial.value == 999.0:
                print(f"Trial {trial.number + 1}: {trial.params}")
    
    # 💾 결과 저장
    best_params_with_details = {
        'best_eval_loss': study.best_value,
        'best_trial_number': study.best_trial.number + 1,
        'best_params': study.best_params,
        'optimization_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_trials': len(study.trials),
        'successful_trials': successful_trials,
        'version': 'final_clean_optuna_only'
    }
    
    with open('optuna_best_params_final.json', 'w', encoding='utf-8') as f:
        json.dump(best_params_with_details, f, indent=2, ensure_ascii=False)
    
    print("\\n💾 결과 저장 완료!")
    print(f"📁 파일: optuna_best_params_final.json")
    print("=" * 60)
    
    # 🎯 다음 단계 안내
    print("\\n🎯 다음 단계:")
    if successful_trials > 0:
        print("1. ✅ 최적 하이퍼파라미터 발견 완료")
        print("2. 🚀 전체 데이터셋으로 최종 학습 실행")
        print("3. 📊 테스트 데이터셋에서 성능 평가")
        print("4. 🏆 삼성 AI 챌린지 제출")
    else:
        print("1. ❌ 모든 Trial 실패 - 디버깅 필요")
        print("2. 🔍 blip_finetune.py 인수 재확인")
        print("3. 🛠️ 데이터 경로 확인")
    
    print(f"\\n🎉 최적화 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

except KeyboardInterrupt:
    print("\\n⛔ 사용자에 의해 중단됨")
    print("💾 현재까지의 결과는 study 객체에 저장되어 있습니다.")
    if 'study' in locals() and len(study.trials) > 0:
        best_trial = min(study.trials, key=lambda t: t.value if t.value is not None else float('inf'))
        print(f"📊 현재까지 최고 성능: {best_trial.value:.4f} (Trial {best_trial.number + 1})")

except Exception as e:
    print(f"\\n❌ 예상치 못한 오류 발생: {e}")
    print("🔍 위의 함수 정의 셀이 실행되었는지 확인하세요.")


[I 2025-07-21 03:46:03,440] A new study created in memory with name: blip-vqa-final-20250721_034603


🎯 Optuna 하이퍼파라미터 최적화 시작!
📊 스터디 이름: blip-vqa-final-20250721_034603
⏰ 10개 실험 순차 실행...
🔧 특징:
  ✅ WandB 완전 제거
  ✅ 모든 인수 오류 해결
  ✅ 환경변수로 WandB 비활성화
  ✅ 빠른 로컬 실행
\n============================================================
\n🎯 Optuna Trial 1:
  - learning_rate: 4.33e-06
  - batch_size: 4
  - num_epochs: 2
  - weight_decay: 0.0143
  - warmup_steps: 300 (ratio: 0.150)
  - 예상 시간: 6분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --logging_steps 20 --eval_strategy epoch --save_strategy no --load_best_model_at_end false --num_train_epochs 2 --output_dir ./optuna-trial-0 --learning_rate 4.328450221293881e-06 --weight_decay 0.01432169828911152 --warmup_ratio 0.15
⏰ GPU 학습 시작...


[I 2025-07-21 04:26:55,457] Trial 0 finished with value: 7.566145420074463 and parameters: {'learning_rate': 4.328450221293881e-06, 'batch_size': 4, 'num_epochs': 2, 'weight_decay': 0.01432169828911152, 'warmup_steps': 300}. Best is trial 0 with value: 7.566145420074463.


✅ eval_loss 찾음: 7.566145420074463
✅ Trial 1 성공!
📊 eval_loss: 7.5661
⏰ 실행 시간: 2452.0초
\n🎯 Optuna Trial 2:
  - learning_rate: 4.44e-05
  - batch_size: 4
  - num_epochs: 2
  - weight_decay: 0.0201
  - warmup_steps: 800 (ratio: 0.400)
  - 예상 시간: 6분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --logging_steps 20 --eval_strategy epoch --save_strategy no --load_best_model_at_end false --num_train_epochs 2 --output_dir ./optuna-trial-1 --learning_rate 4.4447541666908114e-05 --weight_decay 0.02014847788415866 --warmup_ratio 0.4
⏰ GPU 학습 시작...


[I 2025-07-21 07:01:14,363] Trial 1 finished with value: 0.2807295322418213 and parameters: {'learning_rate': 4.4447541666908114e-05, 'batch_size': 4, 'num_epochs': 2, 'weight_decay': 0.02014847788415866, 'warmup_steps': 800}. Best is trial 1 with value: 0.2807295322418213.


✅ eval_loss 찾음: 0.2807295322418213
✅ Trial 2 성공!
📊 eval_loss: 0.2807
⏰ 실행 시간: 9258.9초
\n🎯 Optuna Trial 3:
  - learning_rate: 3.14e-06
  - batch_size: 16
  - num_epochs: 2
  - weight_decay: 0.0327
  - warmup_steps: 500 (ratio: 0.250)
  - 예상 시간: 6분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --logging_steps 20 --eval_strategy epoch --save_strategy no --load_best_model_at_end false --num_train_epochs 2 --output_dir ./optuna-trial-2 --learning_rate 3.1357757322577464e-06 --weight_decay 0.032676417657817626 --warmup_ratio 0.25
⏰ GPU 학습 시작...


In [3]:
# 🔥 Optuna 하이퍼파라미터 최적화 실행 (이어하기 기능 추가)

# study 객체가 없으면 새로 생성
if 'study' not in globals():
    print("✨ 새로운 Optuna study를 생성합니다.")
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),
        study_name=f"blip-vqa-final-resumable-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )

try:
    print(f"✅ 현재 study '{study.study_name}'를 사용합니다.")
    print(f"📊 현재까지 완료된 Trial: {len(study.trials)}개")
        
    # 남은 Trial 계산
    total_trials_goal = 10
    completed_trials = len(study.trials)
    remaining_trials = total_trials_goal - completed_trials

    if remaining_trials > 0:
        print(f"🚀 남은 {remaining_trials}개 Trial을 시작합니다...")
        print("=" * 60)
        study.optimize(optuna_objective, n_trials=remaining_trials)
    else:
        print("✅ 모든 Trial이 이미 완료되었습니다!")

    print("\\n" + "=" * 60)
    print("🎉 Optuna 최적화 완료!")
    print("=" * 60)

    # 🏆 최적 결과 출력
    print(f"✅ 최적 eval_loss: {study.best_value:.4f}")
    print("\\n🎯 최적 하이퍼파라미터:")
    for key, value in study.best_params.items():
        if key == 'learning_rate':
            print(f"  - {key}: {value:.2e}")
        elif key in ['weight_decay']:
            print(f"  - {key}: {value:.4f}")
        else:
            print(f"  - {key}: {value}")
        
    print(f"\\n🏆 최적 Trial 번호: {study.best_trial.number + 1}")
        
    # 📊 전체 결과 요약
    print("\\n📊 전체 실험 결과 요약:")
    print("-" * 60)
    print(f"{'Trial':<8} {'eval_loss':<12} {'lr':<10} {'batch':<8} {'epochs':<8}")
    print("-" * 60)
        
    successful_trials = 0
    for trial in study.trials:
        if trial.value is not None and trial.value < 999.0:
            lr = trial.params.get('learning_rate', 0)
            batch = trial.params.get('batch_size', 0)
            epochs = trial.params.get('num_epochs', 0)
            print(f"{trial.number+1:<8} {trial.value:<12.4f} {lr:<10.2e} {batch:<8} {epochs:<8}")
            successful_trials += 1
        
    if successful_trials == 0:
        print("❌ 성공한 Trial이 없습니다. 디버깅 정보를 확인하세요.")
    
    # 💾 결과 저장
    best_params_with_details = {
        'best_eval_loss': study.best_value,
        'best_trial_number': study.best_trial.number + 1,
        'best_params': study.best_params,
        'optimization_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_trials': len(study.trials),
        'successful_trials': successful_trials,
        'version': 'final_clean_optuna_only'
    }
    
    with open('optuna_best_params_final.json', 'w', encoding='utf-8') as f:
        json.dump(best_params_with_details, f, indent=2, ensure_ascii=False)
    
    print("\\n💾 결과 저장 완료!")
    print(f"📁 파일: optuna_best_params_final.json")
    print("=" * 60)
    
    # 🎯 다음 단계 안내
    print("\\n🎯 다음 단계:")
    if successful_trials > 0:
        print("1. ✅ 최적 하이퍼파라미터 발견 완료")
        print("2. 🚀 전체 데이터셋으로 최종 학습 실행")
        print("3. 📊 테스트 데이터셋에서 성능 평가")
        print("4. 🏆 삼성 AI 챌린지 제출")
    else:
        print("1. ❌ 모든 Trial 실패 - 디버깅 필요")
        print("2. 🔍 blip_finetune.py 인수 재확인")
        print("3. 🛠️ 데이터 경로 확인")
    
    print(f"\\n🎉 최적화 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

except KeyboardInterrupt:
    print("\\n⛔ 사용자에 의해 중단됨")
    print("💾 현재까지의 결과는 study 객체에 저장되어 있습니다.")
    if 'study' in locals() and len(study.trials) > 0:
        best_trial = min(study.trials, key=lambda t: t.value if t.value is not None else float('inf'))
        print(f"📊 현재까지 최고 성능: {best_trial.value:.4f} (Trial {best_trial.number + 1})")

except Exception as e:
    print(f"\\n❌ 예상치 못한 오류 발생: {e}")
    print("🔍 위의 함수 정의 셀이 실행되었는지 확인하세요.")

[I 2025-07-21 17:01:23,173] A new study created in memory with name: blip-vqa-final-resumable-20250721_170123


✨ 새로운 Optuna study를 생성합니다.
✅ 현재 study 'blip-vqa-final-resumable-20250721_170123'를 사용합니다.
📊 현재까지 완료된 Trial: 0개
🚀 남은 10개 Trial을 시작합니다...
\n🎯 Optuna Trial 1:
  - learning_rate: 4.33e-06
  - batch_size: 4
  - num_epochs: 2
  - weight_decay: 0.0143
  - warmup_steps: 300 (ratio: 0.150)
  - 예상 시간: 6분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --logging_steps 20 --eval_strategy epoch --save_strategy no --load_best_model_at_end false --num_train_epochs 2 --output_dir ./optuna-trial-0 --learning_rate 4.328450221293881e-06 --weight_decay 0.01432169828911152 --warmup_ratio 0.15
⏰ GPU 학습 시작...


[I 2025-07-21 17:36:46,779] Trial 0 finished with value: 7.566145420074463 and parameters: {'learning_rate': 4.328450221293881e-06, 'batch_size': 4, 'num_epochs': 2, 'weight_decay': 0.01432169828911152, 'warmup_steps': 300}. Best is trial 0 with value: 7.566145420074463.


✅ eval_loss 찾음: 7.566145420074463
✅ Trial 1 성공!
📊 eval_loss: 7.5661
⏰ 실행 시간: 2123.6초
\n🎯 Optuna Trial 2:
  - learning_rate: 4.44e-05
  - batch_size: 4
  - num_epochs: 2
  - weight_decay: 0.0201
  - warmup_steps: 800 (ratio: 0.400)
  - 예상 시간: 6분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --logging_steps 20 --eval_strategy epoch --save_strategy no --load_best_model_at_end false --num_train_epochs 2 --output_dir ./optuna-trial-1 --learning_rate 4.4447541666908114e-05 --weight_decay 0.02014847788415866 --warmup_ratio 0.4
⏰ GPU 학습 시작...


[I 2025-07-21 18:09:36,234] Trial 1 finished with value: 0.2807295322418213 and parameters: {'learning_rate': 4.4447541666908114e-05, 'batch_size': 4, 'num_epochs': 2, 'weight_decay': 0.02014847788415866, 'warmup_steps': 800}. Best is trial 1 with value: 0.2807295322418213.


...

❌ Trial 8 실패
Return code: 1
STDERR: Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for insta...
\n🎯 Optuna Trial 9:
  - learning_rate: 3.22e-05
  - batch_size: 16
  - num_epochs: 4
  - weight_decay: 0.0364
  - warmup_steps: 100 (ratio: 0.050)
  - 예상 시간: 12분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --logging_step

[I 2025-07-22 04:30:39,759] Trial 8 finished with value: 999.0 and parameters: {'learning_rate': 3.216235469207424e-05, 'batch_size': 16, 'num_epochs': 4, 'weight_decay': 0.036414738668149954, 'warmup_steps': 100}. Best is trial 1 with value: 0.2807295322418213.


❌ Trial 9 실패
Return code: 1
STDERR: Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for insta...
\n🎯 Optuna Trial 10:
  - learning_rate: 1.53e-06
  - batch_size: 8
  - num_epochs: 3
  - weight_decay: 0.0808
  - warmup_steps: 500 (ratio: 0.250)
  - 예상 시간: 9분
🚀 실행 명령어:
c:\Users\Lenovo\anaconda3\envs\2025_Samsung_AI_Challenge\python.exe blip_finetune.py --train_file ../dataset/VQAv2/train.json --val_file ../dataset/VQAv2/val.json --max_train_samples 800 --max_val_samples 400 --per_device_train_batch_size 8 --per_device_eval_batch_size 8 --logging_steps 2

[I 2025-07-22 06:02:40,345] Trial 9 finished with value: 9.408453941345215 and parameters: {'learning_rate': 1.5251209898002929e-06, 'batch_size': 8, 'num_epochs': 3, 'weight_decay': 0.08082886378162035, 'warmup_steps': 500}. Best is trial 1 with value: 0.2807295322418213.


✅ eval_loss 찾음: 9.408453941345215
✅ Trial 10 성공!
📊 eval_loss: 9.4085
⏰ 실행 시간: 5520.6초
\n============================================================
🎉 Optuna 최적화 완료!
✅ 최적 eval_loss: 0.2807
\n🎯 최적 하이퍼파라미터:
  - learning_rate: 4.44e-05
  - batch_size: 4
  - num_epochs: 2
  - weight_decay: 0.0201
  - warmup_steps: 800
\n🏆 최적 Trial 번호: 2
\n📊 전체 실험 결과 요약:
------------------------------------------------------------
Trial    eval_loss    lr         batch    epochs  
------------------------------------------------------------
1        7.5661       4.33e-06   4        2       
2        0.2807       4.44e-05   4        2       
4        0.4243       4.09e-05   4        2       
6        3.7018       3.68e-05   8        2       
10       9.4085       1.53e-06   8        3       
\n💾 결과 저장 완료!
📁 파일: optuna_best_params_final.json
\n🎯 다음 단계:
1. ✅ 최적 하이퍼파라미터 발견 완료
2. 🚀 전체 데이터셋으로 최종 학습 실행
3. 📊 테스트 데이터셋에서 성능 평가
4. 🏆 삼성 AI 챌린지 제출
\n🎉 최적화 완료 시간: 2025-07-22 06:02:40


## 🎉 **완전히 정리된 Optuna 하이퍼파라미터 최적화!**

### ✅ **수정 완료된 문제들:**
1. **WandB 코드 완전 제거** → 순수 Optuna만 사용
2. **`--load_best_model_at_end "false"`** → blip_finetune.py 수정사항 반영
3. **환경변수 WandB 비활성화** → `WANDB_MODE=disabled`
4. **모든 인수 타입 정리** → 문자열 변환 통일
5. **중복 코드 제거** → 깔끔한 3개 셀 구조

### 🚀 **실행 순서:**
1. **셀 1** (환경 확인) → 경로 및 라이브러리 확인
2. **셀 2** (함수 정의) → `optuna_objective` 함수 로드  
3. **셀 3** (Optuna 실행) → **60-90분 최적화 진행**

### ⚡ **실행 전 확인사항:**
- ✅ GPU 환경 활성화
- ✅ 데이터셋 경로 확인: `./dataset/VQAv2/`
- ✅ BLIP 코드 경로 확인: `./BLIP_ViT_L_Finetuning/`

### 🎯 **기대 결과:**
- 📊 **eval_loss 2.0~8.0 범위** (999.0 오류 해결)
- 💾 **optuna_best_params_final.json** 자동 생성
- 🏆 **최적 하이퍼파라미터 자동 발견**

**지금 위의 3개 셀을 순서대로 실행하세요!** 🔥
